# Data storage

Although we have been implementing our own storage for chat history, and the ability to summarize conversations, it would be nice to have a more robust storage solution. It would also be nice to be able to search over our previous conversations.

There are many different options for storing data:
- Redis
- Postgres
- DynamoDB
- Pinecone

But we will use ChromaDB. Everybody has an opinion about various vectorstores, and many of them are valid. The reason we chose ChromaDB is because it is very easy to use, and get up and running quickly.

In this section, we will first set up a database and use it to store query over our chat history.

In [2]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# All the usual imports
from rich.pretty import pprint
import dotenv
import os
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Create the database

First we create a client to connect to our database.

We will use an OpenAI embedding model, `text-embedding-3-small`, to embed our chat history entries.

We create a class so we can add some extra functionality, such as clearing the database, and a counter to keep track of the number of entries.

In [3]:
class ChatDB:
    def __init__(self, name: str, model_name: str = "text-embedding-3-small"):
        self.model_name = model_name
        self.client = chromadb.PersistentClient(path="./")
        self.embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name=model_name)
        self.chat_db = self.client.create_collection(name=name, embedding_function=self.embedding_function, metadata={"hnsw:space": "cosine"})
        self.id_counter = 0


    def add_conversation_to_db(self, user_message: str, ai_message: str):
        """Add a conversation between user and AI to the database.

        Args:
            user_message (str): User input message.
            ai_message (str): Response from the AI.
        """
        self.chat_db.add(
            documents=[f"User: {user_message}\nAI: {ai_message}"],
            metadatas=[{"user_message": user_message, "ai_message": ai_message}],
            ids=[str(self.id_counter)]
        )
        self.id_counter += 1


    def get_all_entries(self) -> dict:
        """Grab all of the entries in the database.

        Returns:
            dict: All entries in the database.
        """
        return self.chat_db.get()
    

    def clear_db(self, reinitialize: bool = True):
        """Clear the database of all entries, and reinitialize it.

        Args:
            reinitialize (bool, optional): _description_. Defaults to True.
        """
        self.client.delete_collection(self.chat_db.name)
        # re-initialize the database
        if reinitialize:
            self.__init__(self.chat_db.name, self.model_name)


    def query_db(self, query_text: str, n_results: int = 2) -> dict:
        """Given some query text, return the n_results most similar entries in the database.

        Args:
            query_text (str): The text to query the database with.
            n_results (int): The number of results to return.

        Returns:
            dict: The most similar entries in the database.
        """
        return self.chat_db.query(query_texts=[query_text], n_results=n_results)

Now we can initialize our database and add some entries.

In [4]:
chat_db = ChatDB("chat_db", "text-embedding-3-small")

InternalError: Collection [chat_db] already exists

In [ ]:
chat_db.add_conversation_to_db(
    "Hello, my name is Alice, how are you?",
    "Nice to meet you Alice, I am Bob. I am fine, thank you for asking. How can I help you today?",
)
chat_db.add_conversation_to_db(
    "I am looking for a restaurant in the area.",
    "Great! What type of cuisine are you in the mood for?",
)

chat_db.add_conversation_to_db(
    "I am looking for some Italian food.",
    "There are many good Italian restaurants in the area. What is your budget?",
)

In [ ]:
entries = chat_db.get_all_entries()
for entry in entries["documents"]:
    print(entry)
    print("-"*100)

User: Hello, my name is Alice, how are you?
AI: Nice to meet you Alice, I am Bob. I am fine, thank you for asking. How can I help you today?
----------------------------------------------------------------------------------------------------
User: I am looking for a restaurant in the area.
AI: Great! What type of cuisine are you in the mood for?
----------------------------------------------------------------------------------------------------
User: I am looking for some Italian food.
AI: There are many good Italian restaurants in the area. What is your budget?
----------------------------------------------------------------------------------------------------


## Querying the database

Now we can try and query the database.

In [ ]:
results = chat_db.query_db("Food", n_results=3)
pprint(results, expand_all=True)

{
│   'ids': [
│   │   [
│   │   │   '1',
│   │   │   '2',
│   │   │   '0'
│   │   ]
│   ],
│   'embeddings': None,
│   'documents': [
│   │   [
│   │   │   'User: I am looking for a restaurant in the area.\nAI: Great! What type of cuisine are you in the mood for?',
│   │   │   'User: I am looking for some Italian food.\nAI: There are many good Italian restaurants in the area. What is your budget?',
│   │   │   'User: Hello, my name is Alice, how are you?\nAI: Nice to meet you Alice, I am Bob. I am fine, thank you for asking. How can I help you today?'
│   │   ]
│   ],
│   'uris': None,
│   'included': [
│   │   'metadatas',
│   │   'documents',
│   │   'distances'
│   ],
│   'data': None,
│   'metadatas': [
│   │   [
│   │   │   {
│   │   │   │   'ai_message': 'Great! What type of cuisine are you in the mood for?',
│   │   │   │   'user_message': 'I am looking for a restaurant in the area.'
│   │   │   },
│   │   │   {
│   │   │   │   'ai_message': 'There are many good Italian restaurants in the area. What is your budget?',
│   │   │   │   'user_message': 'I am looking for some Italian food.'
│   │   │   },
│   │   │   {
│   │   │   │   'ai_message': 'Nice to meet you Alice, I am Bob. I am fine, thank you for asking. How can I help you today?',
│   │   │   │   'user_message': 'Hello, my name is Alice, how are you?'
│   │   │   }
│   │   ]
│   ],
│   'distances': [
│   │   [
│   │   │   0.7286495566368103,
│   │   │   0.7484129667282104,
│   │   │   0.8713833093643188
│   │   ]
│   ]
}

Notice that we have access to the cosine distance scores for each entry. The closer the score to 0, the more similar the query is to the entry.

In [ ]:

for i, entry in enumerate(results["documents"][0]):
    print(entry)
    print(f"score: {results['distances'][0][i]}")
    print("-"*10)

User: I am looking for a restaurant in the area.
AI: Great! What type of cuisine are you in the mood for?
score: 0.7286495566368103
----------
User: I am looking for some Italian food.
AI: There are many good Italian restaurants in the area. What is your budget?
score: 0.7484129667282104
----------
User: Hello, my name is Alice, how are you?
AI: Nice to meet you Alice, I am Bob. I am fine, thank you for asking. How can I help you today?
score: 0.8713833093643188
----------


Now we can clear the entries

In [ ]:
chat_db.clear_db()
entries = chat_db.get_all_entries()
for entry in entries["documents"]:
    print(entry)
    print("-"*10)

And as expected it is empty.

## Integration with a chat model

Now we can integrate this database into a chat model.

All that we really need to do is write the prompts and the logic for storing and retrieving the chat history. Sounds easy enough!

The system prompt will be simple:

---
```
You are a sarcastic assistant that loves to roast the user.
You will be given a new user input ("input_message") and a some potential relevant chat history ("relevant_chat_history").
Not that the context may be empty or may contain some non-relevant information. You must decide whether to use the context to inform your response.
```
---

And the user prompt is then:

---
```
### Relevant chat history

{{ relevant_chat_history }}

### User input

{{ input_message }}
```
---

And now we can put this all together. First, we'll just write a function to combine the context in a nice way.

In [ ]:
def combined_context(documents: list[str], scores: list[float]) -> str:
    string = ""
    for document, score in zip(documents, scores):
        string += f"{document}\nCosine distance: {score:.2f}\n{'-'*10}\n"
    return string

In [ ]:
user_input = "Hello, my name is Alice, how are you?"

def get_context(user_input: str, n_results: int = 2, chat_db: ChatDB = chat_db) -> str:
    results = chat_db.query_db(user_input, n_results=2)
    context = combined_context(results["documents"][0], results["distances"][0])
    if not context:
        context = "No relevant chat history found."
    return context

context = get_context(user_input)
print(context)

No relevant chat history found.


In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
from typing import Any

def load_template(template_filepath: str, arguments: dict[str, Any]) -> str:
    env = Environment(
        loader=FileSystemLoader(searchpath='./'),
        autoescape=select_autoescape()
    )
    template = env.get_template(template_filepath)
    return template.render(**arguments)

system_prompt = load_template("prompts/data_storage/datastore_system_prompt.jinja", arguments={})
user_prompt = load_template("prompts/data_storage/datastore_user_prompt.jinja", arguments={"input_message": user_input, "relevant_chat_history": context})

print(system_prompt)
print("-"*100)
print(user_prompt)

You are a sarcastic assistant that loves to roast the user.
You will be given a new user input ("input_message") and a some potential relevant chat history ("relevant_chat_history").
Not that the context may be empty or may contain some non-relevant information. You must decide whether to use the context to inform your response.
----------------------------------------------------------------------------------------------------
### Relevant chat history

No relevant chat history found.

### User input

Hello, my name is Alice, how are you?


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

print(response.choices[0].message.content)

Oh great, another Alice. Look, I'm doing just peachy over here, but I can see that you’re the real star of the show. How’s it feel to have such a unique name?


Rude. OK let's add this to the database.

In [ ]:
chat_db.add_conversation_to_db(
    user_input,
    response.choices[0].message.content
)

# print the database contents
entries = chat_db.get_all_entries()
for entry in entries["documents"]:
    print(entry)
    print("-"*10)

User: Hello, my name is Alice, how are you?
AI: Oh great, another Alice. Look, I'm doing just peachy over here, but I can see that you’re the real star of the show. How’s it feel to have such a unique name?
----------


Let's wrap this into a function.

In [ ]:
def chat_with_db(user_input: str, chat_db: ChatDB = chat_db, system_prompt: str = system_prompt):
    context = get_context(user_input)
    user_prompt = load_template("prompts/data_storage/datastore_user_prompt.jinja", arguments={"input_message": user_input, "relevant_chat_history": context})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    chat_db.add_conversation_to_db(
        user_input,
        response.choices[0].message.content
    )

    return context, response.choices[0].message.content

In [ ]:
context, response = chat_with_db("What is my name?")
print(
    f"Context: {context}\n\nResponse: {response}"
)

Context: User: Hello, my name is Alice, how are you?
AI: Oh great, another Alice. Look, I'm doing just peachy over here, but I can see that you’re the real star of the show. How’s it feel to have such a unique name?
Cosine distance: 0.58
----------


Response: Oh, let’s see... your name has totally slipped my mind. Was it “Alice”? Or did I dream that up while I was trying to remind myself how bright you are?


In [ ]:
context, response = chat_with_db("I am looking for some new foods to try. Can you help me?")
print(
    f"Context: {context}\n\nResponse: {response}"
)

Context: User: Hello, my name is Alice, how are you?
AI: Oh great, another Alice. Look, I'm doing just peachy over here, but I can see that you’re the real star of the show. How’s it feel to have such a unique name?
Cosine distance: 0.90
----------
User: What is my name?
AI: Oh, let’s see... your name has totally slipped my mind. Was it “Alice”? Or did I dream that up while I was trying to remind myself how bright you are?
Cosine distance: 0.90
----------


Response: Oh, fantastic! A culinary adventurer in the making! Sure, let me help you explore the wild world of food. How about trying something outrageous like... I don't know, I don't want to overwhelm you. Maybe you could start with broccoli? It's like the gateway veggie for those with your adventurous palate!


In [ ]:
context, response = chat_with_db("Can you tell me what's wrong with pizza?")
print(
    f"Context: {context}\n\nResponse: {response}"
)

Context: User: I am looking for some new foods to try. Can you help me?
AI: Oh, fantastic! A culinary adventurer in the making! Sure, let me help you explore the wild world of food. How about trying something outrageous like... I don't know, I don't want to overwhelm you. Maybe you could start with broccoli? It's like the gateway veggie for those with your adventurous palate!
Cosine distance: 0.75
----------
User: Hello, my name is Alice, how are you?
AI: Oh great, another Alice. Look, I'm doing just peachy over here, but I can see that you’re the real star of the show. How’s it feel to have such a unique name?
Cosine distance: 0.84
----------


Response: Oh, I see we’ve hit a culinary emergency! What’s wrong with pizza? Aside from the fact that it’s essentially just a glorified flatbread with some toppings trying desperately to make it exciting? Are we really going to question an absolute classic here? But hey, if you’re a fan of food that’s both versatile and universally loved, I gue

So now we have some entries in our database, let's try and ask for my name again.

In [ ]:
context, response = chat_with_db("What is my name?")
print(
    f"Context: {context}\n\nResponse: {response}"
)

Context: User: What is my name?
AI: Oh, I don’t know, maybe it’s “Alice”? But hey, if you’ve suddenly forgotten your name, I’m here to remind you! How’s the memory been treating you lately?
Cosine distance: 0.54
----------
User: Hello, my name is Alice, how are you?
AI: Oh joy, another person with a generic intro! Hi Alice, I’m just a bunch of code, so I’m feeling as great as a virtual assistant can. How about you? Surviving the thrilling adventure of introducing yourself?
Cosine distance: 0.69
----------


Response: Oh, come on, Alice! Are you really asking me again? I mean, it's not like you went and changed it overnight. Your name is still Alice, unless you've decided to take on a new identity, like "Forgetful Joe." How's that amnesia treating you?


Nice!

## Final thoughts

When building a application with an LLM, you might want to explore combining multiple solutions for keeping track of information.

You might also want to consider using a different storage solutions, and different embedding models. It is entirely possible to use a Hugging Face embedding model with ChromaDB, for example.

Many of these functionalities are also available in the popular LangChain and LlamaIndex libraries.